# OilWell Location Optimizer — Risk Pipeline End-to-End

Este notebook ejecuta de forma encadenada el pipeline principal de riesgo:

1. Entrenamiento de modelos por región (`main.py --mode train`).
2. Evaluación de riesgo con bootstrap (`main.py --mode eval`).
3. Lectura de métricas y recomendación de región/pozos a partir de `artifacts/metrics.json` y `artifacts/risk_results.json`.


In [ ]:
from pathlib import Path

import json
import yaml

from main import load_config, cmd_train, cmd_eval


In [ ]:
# Cargar configuración por defecto
base = Path('.').resolve()
cfg_path = base / 'configs' / 'default.yaml'
cfg_dict = yaml.safe_load(cfg_path.read_text())

print('Config file:', cfg_path)
cfg_dict


In [ ]:
# 1) Entrenamiento (vía cmd_train)

print('== Entrenando modelos por región ==')
cmd_train(cfg_dict, base)
print('Entrenamiento completado. Revisa artifacts/metrics.json')


In [ ]:
# 2) Evaluación de riesgo vía bootstrap

print('== Evaluando riesgo por región (bootstrap) ==')
seed = cfg_dict.get('project', {}).get('seed', 42)
cmd_eval(cfg_dict, base, seed=seed)
print('Evaluación de riesgo completada. Revisa artifacts/risk_results.json')


In [ ]:
# 3) Lectura de métricas y recomendación de región

metrics_path = base / 'artifacts' / 'metrics.json'
risk_path = base / 'artifacts' / 'risk_results.json'

metrics = json.loads(metrics_path.read_text()) if metrics_path.exists() else {}
risk = json.loads(risk_path.read_text()) if risk_path.exists() else {}

print('Métricas de entrenamiento por región:')
display(metrics)

print('Resultados de riesgo por región:')
display(risk)

# Recomendar región con mayor beneficio esperado y riesgo bajo
regions = risk.get('regions', risk.get('regions', {})) if isinstance(risk, dict) else {}
best_region = None
best_profit = None
for region, stats in regions.items():
    profit = stats.get('expected_profit')
    loss_prob = stats.get('loss_probability')
    if profit is None or loss_prob is None:
        continue
    if loss_prob > 0.025:  # criterio de ejemplo: riesgo de pérdida < 2.5%
        continue
    if best_profit is None or profit > best_profit:
        best_profit = profit
        best_region = region

print('
Recomendación de ejemplo (sujeta a criterios / supuestos):')
print('Región recomendada:', best_region)
print('Beneficio esperado (aprox.):', best_profit)


## Notas

- Este notebook reutiliza las funciones `cmd_train` y `cmd_eval` definidas en `main.py` y los parámetros de `configs/default.yaml`.
- El criterio de selección de región es ilustrativo (beneficio esperado alto con probabilidad de pérdida <2.5%).
- En un entorno real se deberían incorporar criterios adicionales (constraints operativos, límites regulatorios, etc.).
